# Introducción a MLflow 🤖

## 📚 Objetivos de Aprendizaje

En este notebook aprenderás:
1. ¿Qué es MLflow y por qué lo necesitas?
2. Los 4 componentes principales de MLflow
3. MLflow Tracking: Rastrear experimentos
4. MLflow Models: Empaquetar y versionar modelos
5. MLflow Model Registry: Gestión del ciclo de vida
6. Comandos y funciones esenciales
7. Best practices

---

## 🎯 ¿Qué es MLflow?

**MLflow** es una plataforma open-source para gestionar el ciclo de vida completo de Machine Learning.

### El Problema que Resuelve:

Sin MLflow:
```
❌ "¿Qué hiperparámetros usé en ese experimento?"
❌ "¿Cuál era la versión del modelo en producción?"
❌ "No puedo reproducir los resultados de la semana pasada"
❌ "¿Dónde guardé ese modelo?"
❌ "¿Qué datos usé para entrenar?"
```

Con MLflow:
```
✅ Todos los experimentos rastreados automáticamente
✅ Modelos versionados y organizados
✅ Reproducibilidad garantizada
✅ Comparación fácil entre experimentos
✅ Deployment simplificado
```

### ¿Para quién es MLflow?

- **Data Scientists**: Experimentación y desarrollo de modelos
- **ML Engineers**: Deployment y gestión de modelos
- **Equipos**: Colaboración y compartir modelos
- **Organizaciones**: Gobernanza y compliance

---

## 🏗️ Los 4 Componentes de MLflow

```
┌─────────────────────────────────────────────────────────┐
│                       MLFLOW                            │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  1️⃣ TRACKING                    2️⃣ PROJECTS          │
│  Rastrea experimentos           Empaqueta código       │
│  - Parámetros                   - Reproducible         │
│  - Métricas                     - Portable             │
│  - Artefactos                   - Versionado           │
│                                                         │
│  3️⃣ MODELS                      4️⃣ MODEL REGISTRY    │
│  Empaqueta modelos              Gestiona ciclo de vida │
│  - Formato estándar             - Staging/Production   │
│  - Multi-framework              - Versionado           │
│  - Deployment fácil             - Colaboración         │
│                                                         │
└─────────────────────────────────────────────────────────┘
```

### 1️⃣ MLflow Tracking
Registra y consulta experimentos:
- Parámetros (learning rate, epochs, etc.)
- Métricas (accuracy, loss, etc.)
- Modelos y artefactos
- Código fuente y versiones

### 2️⃣ MLflow Projects
Formato para empaquetar código de ML:
- Dependencias explícitas
- Puntos de entrada definidos
- Parámetros configurables

### 3️⃣ MLflow Models
Formato estándar para modelos:
- Funciona con sklearn, TensorFlow, PyTorch, etc.
- Incluye metadata y dependencias
- Deploy en múltiples plataformas

### 4️⃣ MLflow Model Registry
Repositorio centralizado de modelos:
- Versionado automático
- Transiciones de stage (Staging → Production)
- Anotaciones y descripciones
- Control de acceso

---

## ⚙️ Setup y Configuración

In [ ]:
# Imports
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient

import numpy as np
import pandas as pd
from datetime import datetime

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.datasets import make_classification

print("✅ Librerías importadas")
print(f"📦 MLflow version: {mlflow.__version__}")

In [ ]:
# Configurar MLflow
# En Databricks, MLflow ya está integrado

# Obtener usuario actual
username = spark.sql("SELECT current_user()").first()[0]

# Nombre del experimento (cada usuario tiene el suyo)
experiment_name = f"/Users/{username}/mlflow_intro_demo"

# Crear o usar experimento existente
mlflow.set_experiment(experiment_name)

print(f"🔬 Experimento configurado: {experiment_name}")
print(f"\n📍 Puedes ver tus experimentos en:")
print(f"   Machine Learning > Experiments en el menú lateral")

---

## 📊 Preparar Datos de Ejemplo

Usaremos un dataset sintético de clasificación binaria para demostrar MLflow.

In [ ]:
# Generar dataset sintético
X, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=8,
    n_redundant=2,
    n_classes=2,
    random_state=42
)

# Crear DataFrame para mejor visualización
feature_names = [f'feature_{i}' for i in range(X.shape[1])]
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y

print(f"✅ Dataset creado:")
print(f"   Muestras: {len(df):,}")
print(f"   Features: {X.shape[1]}")
print(f"   Clases: {len(np.unique(y))}")
print(f"\nDistribución de clases:")
print(df['target'].value_counts())

df.head()

In [ ]:
# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"✅ Datos divididos:")
print(f"   Training: {len(X_train):,} samples")
print(f"   Test: {len(X_test):,} samples")

---

## 1️⃣ MLflow Tracking: Conceptos Básicos

### Jerarquía de MLflow:

```
Experiment (Experimento)
  └── Run (Ejecución)
       ├── Parameters (Hiperparámetros)
       ├── Metrics (Métricas)
       ├── Tags (Etiquetas)
       ├── Artifacts (Artefactos: modelos, plots, etc.)
       └── Model (Modelo empaquetado)
```

### Conceptos Clave:

- **Experiment**: Colección de runs relacionados (ej: "Clasificador de Clientes")
- **Run**: Un intento de entrenamiento (ej: "Random Forest con max_depth=10")
- **Parameters**: Configuración del modelo (ej: learning_rate=0.01)
- **Metrics**: Resultados del modelo (ej: accuracy=0.95)
- **Artifacts**: Archivos generados (ej: modelo.pkl, plot.png)
- **Tags**: Metadata adicional (ej: "producción", "experimental")

### 🔥 Tu Primer Run en MLflow

In [ ]:
# EJEMPLO BÁSICO: Entrenar modelo y rastrear con MLflow

# Iniciar un run
with mlflow.start_run(run_name="mi_primer_modelo") as run:
    
    # 1. Log parámetros
    mlflow.log_param("modelo", "Logistic Regression")
    mlflow.log_param("max_iter", 1000)
    mlflow.log_param("solver", "lbfgs")
    
    # 2. Entrenar modelo
    model = LogisticRegression(max_iter=1000, solver='lbfgs', random_state=42)
    model.fit(X_train, y_train)
    
    # 3. Evaluar
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # 4. Log métricas
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    
    # 5. Log modelo
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        registered_model_name="demo_classifier"
    )
    
    # 6. Log tags
    mlflow.set_tag("tipo", "clasificacion")
    mlflow.set_tag("framework", "sklearn")
    
    print("✅ Run completado!")
    print(f"\n📊 Métricas:")
    print(f"   Accuracy:  {accuracy:.4f}")
    print(f"   Precision: {precision:.4f}")
    print(f"   Recall:    {recall:.4f}")
    print(f"   F1-Score:  {f1:.4f}")
    print(f"\n🔗 Run ID: {run.info.run_id}")

### 📝 Comandos Esenciales de MLflow Tracking

#### Iniciar/Terminar Runs:

```python
# Opción 1: Context manager (recomendado)
with mlflow.start_run(run_name="mi_experimento") as run:
    # Tu código aquí
    pass

# Opción 2: Manual
run = mlflow.start_run()
# Tu código
mlflow.end_run()
```

#### Registrar Parámetros:

```python
# Un parámetro
mlflow.log_param("learning_rate", 0.01)

# Múltiples parámetros
mlflow.log_params({
    "learning_rate": 0.01,
    "epochs": 100,
    "batch_size": 32
})
```

#### Registrar Métricas:

```python
# Una métrica
mlflow.log_metric("accuracy", 0.95)

# Múltiples métricas
mlflow.log_metrics({
    "accuracy": 0.95,
    "loss": 0.23,
    "f1_score": 0.92
})

# Métrica con steps (para gráficos de entrenamiento)
for epoch in range(10):
    mlflow.log_metric("loss", loss_value, step=epoch)
```

#### Registrar Artefactos:

```python
# Archivo individual
mlflow.log_artifact("plot.png")

# Directorio completo
mlflow.log_artifacts("./outputs")

# Crear y guardar texto
with open("readme.txt", "w") as f:
    f.write("Descripción del modelo")
mlflow.log_artifact("readme.txt")
```

#### Tags:

```python
# Un tag
mlflow.set_tag("version", "v1.0")

# Múltiples tags
mlflow.set_tags({
    "team": "data-science",
    "priority": "high"
})
```

### 🔄 Autolog: Tracking Automático

MLflow puede rastrear automáticamente parámetros, métricas y modelos según el framework.

In [ ]:
# AUTOLOG DEMO

# Activar autolog para sklearn
mlflow.sklearn.autolog()

with mlflow.start_run(run_name="autolog_demo") as run:
    
    # Solo entrenamos - MLflow registra todo automáticamente
    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42
    )
    model.fit(X_train, y_train)
    
    # Hacer predicción para que autolog calcule métricas
    y_pred = model.predict(X_test)
    
    print("✅ Modelo entrenado con autolog")
    print("\n🤖 MLflow registró automáticamente:")
    print("   - Todos los parámetros del modelo")
    print("   - Feature importances")
    print("   - Modelo serializado")
    print("   - Signature del modelo")
    print(f"\n🔗 Run ID: {run.info.run_id}")

# Desactivar autolog
mlflow.sklearn.autolog(disable=True)

### 🔍 Comparar Múltiples Experimentos

Una de las ventajas clave de MLflow es comparar diferentes configuraciones.

In [ ]:
# Probar diferentes configuraciones de Random Forest

configurations = [
    {"n_estimators": 50, "max_depth": 5},
    {"n_estimators": 100, "max_depth": 10},
    {"n_estimators": 200, "max_depth": 15},
    {"n_estimators": 100, "max_depth": None},
]

results = []

for i, config in enumerate(configurations):
    with mlflow.start_run(run_name=f"random_forest_config_{i+1}") as run:
        
        # Log parámetros
        mlflow.log_params(config)
        
        # Entrenar
        model = RandomForestClassifier(**config, random_state=42)
        model.fit(X_train, y_train)
        
        # Evaluar
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # Log métricas
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("f1_score", f1)
        
        # Guardar resultados para comparación
        results.append({
            "config": f"Config {i+1}",
            "n_estimators": config["n_estimators"],
            "max_depth": config["max_depth"],
            "accuracy": accuracy,
            "f1_score": f1,
            "run_id": run.info.run_id
        })
        
        print(f"✅ Config {i+1}: Accuracy={accuracy:.4f}, F1={f1:.4f}")

# Comparar resultados
results_df = pd.DataFrame(results)
print("\n📊 COMPARACIÓN DE CONFIGURACIONES:\n")
display(results_df.sort_values("accuracy", ascending=False))

### 🔎 Buscar y Consultar Runs

MLflow proporciona APIs para buscar y filtrar runs.

In [ ]:
# Obtener todos los runs del experimento actual
experiment = mlflow.get_experiment_by_name(experiment_name)

# Buscar runs
runs = mlflow.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["metrics.accuracy DESC"],  # Ordenar por accuracy
    max_results=10
)

print("🔍 Runs encontrados (ordenados por accuracy):")
display(runs[[
    'run_id',
    'tags.mlflow.runName',
    'metrics.accuracy',
    'metrics.f1_score',
    'params.n_estimators',
    'params.max_depth'
]].head())

In [ ]:
# Filtrar runs con condiciones
high_accuracy_runs = mlflow.search_runs(
    experiment_ids=[experiment.experiment_id],
    filter_string="metrics.accuracy > 0.85",  # Solo runs con accuracy > 85%
    order_by=["metrics.f1_score DESC"]
)

print(f"\n🎯 Runs con accuracy > 0.85: {len(high_accuracy_runs)}")
if len(high_accuracy_runs) > 0:
    display(high_accuracy_runs[[
        'tags.mlflow.runName',
        'metrics.accuracy',
        'metrics.f1_score'
    ]].head())

---

## 2️⃣ MLflow Models: Empaquetar Modelos

MLflow Models es un formato estándar para empaquetar modelos de ML de cualquier framework.

### Ventajas:

- ✅ **Framework agnóstico**: sklearn, TensorFlow, PyTorch, etc.
- ✅ **Incluye dependencias**: requirements.txt automático
- ✅ **Signature**: Define input/output del modelo
- ✅ **Múltiples formatos**: Python function, REST API, etc.
- ✅ **Deployment fácil**: Una línea de código

In [ ]:
# Guardar modelo con signature

with mlflow.start_run(run_name="modelo_con_signature") as run:
    
    # Entrenar
    model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    model.fit(X_train, y_train)
    
    # Crear signature (define input/output)
    signature = infer_signature(X_train, model.predict(X_train))
    
    # Input example
    input_example = X_train[:5]
    
    # Log modelo con metadata completa
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        signature=signature,
        input_example=input_example,
        registered_model_name="demo_rf_classifier"
    )
    
    print("✅ Modelo guardado con:")
    print("   - Signature (input/output schema)")
    print("   - Input example")
    print("   - Dependencias (conda.yaml, requirements.txt)")
    print(f"\n🔗 Run ID: {run.info.run_id}")

In [ ]:
# Cargar modelo guardado

# Opción 1: Desde un run específico
run_id = run.info.run_id
model_uri = f"runs:/{run_id}/model"
loaded_model = mlflow.sklearn.load_model(model_uri)

# Hacer predicciones
predictions = loaded_model.predict(X_test[:5])

print("✅ Modelo cargado desde run")
print(f"\nPredicciones de ejemplo: {predictions}")
print(f"Valores reales: {y_test[:5]}")

### 📋 Formatos de Modelo MLflow

MLflow puede guardar modelos en múltiples formatos:

```python
# Scikit-learn
mlflow.sklearn.log_model(model, "model")

# PyTorch
mlflow.pytorch.log_model(model, "model")

# TensorFlow/Keras
mlflow.tensorflow.log_model(model, "model")
mlflow.keras.log_model(model, "model")

# XGBoost
mlflow.xgboost.log_model(model, "model")

# LightGBM
mlflow.lightgbm.log_model(model, "model")

# Python function (custom)
mlflow.pyfunc.log_model(python_model=custom_model, artifact_path="model")
```

---

## 3️⃣ MLflow Model Registry: Gestión del Ciclo de Vida

El Model Registry es un repositorio centralizado para gestionar modelos.

### Stages del Ciclo de Vida:

```
None (Nuevo) → Staging (Testing) → Production (Activo) → Archived (Deprecado)
```

### Flujo Típico:

```
1. Data Scientist entrena modelo
   ↓
2. Modelo se registra automáticamente (None)
   ↓
3. Se promociona a Staging para testing
   ↓
4. Después de validación, va a Production
   ↓
5. Modelo antiguo se archiva
```

In [ ]:
# Trabajar con el Model Registry

client = MlflowClient()

# Nombre del modelo registrado
model_name = "demo_rf_classifier"

# Obtener todas las versiones del modelo
versions = client.search_model_versions(f"name='{model_name}'")

print(f"📦 Modelo: {model_name}")
print(f"📊 Total de versiones: {len(versions)}\n")

for version in versions[:3]:  # Mostrar primeras 3
    print(f"Versión {version.version}:")
    print(f"  Stage: {version.current_stage}")
    print(f"  Run ID: {version.run_id}")
    print()

In [ ]:
# Transicionar modelo a diferentes stages

# Obtener la última versión
latest_version = versions[0].version if versions else 1

print(f"🔄 Transicionando versión {latest_version} a Staging...")

# Mover a Staging
client.transition_model_version_stage(
    name=model_name,
    version=latest_version,
    stage="Staging",
    archive_existing_versions=False  # No archivar otras versiones en Staging
)

print("✅ Modelo en Staging")

# Agregar descripción
client.update_model_version(
    name=model_name,
    version=latest_version,
    description="Random Forest classifier entrenado con 100 árboles y max_depth=10"
)

print("✅ Descripción agregada")

In [ ]:
# Cargar modelo desde Model Registry

# Opción 1: Por stage
model_staging = mlflow.sklearn.load_model(f"models:/{model_name}/Staging")

# Opción 2: Por versión específica
model_v1 = mlflow.sklearn.load_model(f"models:/{model_name}/{latest_version}")

print("✅ Modelos cargados desde Registry")
print(f"   - Modelo en Staging")
print(f"   - Modelo versión {latest_version}")

# Hacer predicción
test_sample = X_test[:1]
prediction = model_staging.predict(test_sample)
print(f"\n🔮 Predicción de ejemplo: {prediction[0]}")

In [ ]:
# Simular promoción a Production

print("🚀 Simulando validación y promoción a Production...\n")

# En la realidad, aquí harías:
# - A/B testing
# - Validación con datos recientes
# - Revisión por equipo de ML

# Promocionar a Production
client.transition_model_version_stage(
    name=model_name,
    version=latest_version,
    stage="Production",
    archive_existing_versions=True  # Archivar versiones anteriores en Production
)

print(f"✅ Versión {latest_version} ahora en PRODUCTION")
print("📦 Versiones anteriores archivadas")

# Agregar tag
client.set_model_version_tag(
    name=model_name,
    version=latest_version,
    key="deployment_date",
    value=datetime.now().strftime("%Y-%m-%d")
)

print("✅ Tag de deployment agregado")

### 🔧 Comandos del Model Registry

```python
from mlflow.tracking import MlflowClient
client = MlflowClient()

# Registrar modelo
mlflow.register_model(
    model_uri="runs:/run_id/model",
    name="my_model"
)

# Buscar versiones
versions = client.search_model_versions("name='my_model'")

# Obtener última versión
latest_versions = client.get_latest_versions("my_model", stages=["Production"])

# Transicionar stage
client.transition_model_version_stage(
    name="my_model",
    version=1,
    stage="Production"
)

# Actualizar descripción
client.update_model_version(
    name="my_model",
    version=1,
    description="Descripción del modelo"
)

# Agregar tags
client.set_model_version_tag(
    name="my_model",
    version=1,
    key="team",
    value="data-science"
)

# Eliminar versión (solo si no está en Production)
client.delete_model_version(
    name="my_model",
    version=1
)
```

---

## 📊 Visualización en la UI de MLflow

### Acceder a la UI:

En Databricks:
1. Menú lateral → **Machine Learning**
2. Click en **Experiments**
3. Encuentra tu experimento

### Qué puedes hacer en la UI:

#### Experiments View:
- ✅ Ver todos los runs de un experimento
- ✅ Comparar runs lado a lado
- ✅ Visualizar métricas en gráficos
- ✅ Filtrar y buscar runs
- ✅ Descargar artefactos

#### Run Details:
- ✅ Parámetros y métricas
- ✅ Gráficos de métricas vs steps
- ✅ Artefactos (modelos, plots, etc.)
- ✅ Código fuente
- ✅ Información del sistema

#### Model Registry:
- ✅ Ver todas las versiones
- ✅ Comparar versiones
- ✅ Gestionar stages
- ✅ Ver requests de transición
- ✅ Webhooks y notificaciones

---

## 🎯 Best Practices

### 1. Organización de Experimentos:

```
✅ BIEN:
/Users/tu_usuario/
  ├── customer_churn_prediction/
  ├── fraud_detection/
  └── recommendation_system/

❌ MAL:
/Users/tu_usuario/
  ├── experiment_1
  ├── test
  └── new_model
```

### 2. Nombres de Runs:

```python
# ✅ Descriptivos
run_name = "rf_100trees_depth10_2024-01-15"
run_name = "logistic_baseline_v1"

# ❌ Poco informativos
run_name = "test"
run_name = "run1"
```

### 3. Qué Registrar:

**SIEMPRE registra:**
- ✅ Todos los hiperparámetros
- ✅ Métricas de evaluación
- ✅ Información del dataset (tamaño, features, etc.)
- ✅ Versión del código/modelo

**Considera registrar:**
- 📊 Plots de performance
- 📊 Confusion matrices
- 📊 Feature importances
- 📊 Información del cluster/ambiente

### 4. Tags Útiles:

```python
mlflow.set_tags({
    "team": "data-science",
    "project": "customer-retention",
    "model_type": "classification",
    "status": "experimental",
    "git_commit": "abc123",
    "author": "tu_nombre"
})
```

### 5. Model Registry:

```
✅ Una sola fuente de verdad
✅ Siempre usar stages (None → Staging → Production)
✅ Describir cambios en cada versión
✅ Archivar modelos obsoletos
✅ Tags para metadata adicional
```

### 6. Limpieza:

```python
# Eliminar runs experimentales fallidos
client.delete_run(run_id)

# Archivar experimentos viejos
# (No hay delete de experimentos en Databricks MLflow)
```

---

## 🎓 Ejercicios Prácticos

### Ejercicio 1: Experimentación Básica
1. Entrena 3 modelos diferentes (ej: Logistic Regression, Random Forest, Gradient Boosting)
2. Registra parámetros, métricas y modelos con MLflow
3. Compara los resultados en la UI
4. Identifica el mejor modelo

### Ejercicio 2: Hyperparameter Tuning
1. Usa un loop para probar diferentes configuraciones de hiperparámetros
2. Registra cada configuración como un run separado
3. Usa `mlflow.search_runs()` para encontrar la mejor configuración
4. Visualiza las métricas vs parámetros

### Ejercicio 3: Model Registry Workflow
1. Registra tu mejor modelo en el Model Registry
2. Transiónalo a Staging
3. Carga el modelo desde Staging y haz predicciones
4. Agrégale descripción y tags
5. Promociónalo a Production

### Ejercicio 4: Autolog
1. Activa autolog para sklearn
2. Entrena un modelo complejo (ej: GridSearchCV)
3. Revisa qué registró automáticamente MLflow
4. Compara con un run manual

### Ejercicio 5: Custom Artifacts
1. Crea un plot de confusion matrix
2. Guárdalo como imagen
3. Regístralo como artefacto con `mlflow.log_artifact()`
4. Crea un archivo de texto con el classification report
5. Regístralo también

In [ ]:
# ESPACIO PARA TUS EJERCICIOS

# Ejercicio 1: Tu código aquí


---

## 📚 Comandos de Referencia Rápida

### Setup y Configuración:
```python
import mlflow
mlflow.set_experiment("/Users/usuario/mi_experimento")
```

### Tracking:
```python
with mlflow.start_run(run_name="mi_run") as run:
    mlflow.log_param("param", value)
    mlflow.log_params({"p1": v1, "p2": v2})
    mlflow.log_metric("metric", value)
    mlflow.log_metrics({"m1": v1, "m2": v2})
    mlflow.log_artifact("file.txt")
    mlflow.set_tag("tag", "value")
```

### Models:
```python
# Guardar
mlflow.sklearn.log_model(model, "model")

# Cargar desde run
model = mlflow.sklearn.load_model(f"runs:/{run_id}/model")

# Cargar desde registry
model = mlflow.sklearn.load_model("models:/modelo/Production")
```

### Model Registry:
```python
from mlflow.tracking import MlflowClient
client = MlflowClient()

# Registrar
mlflow.register_model("runs:/run_id/model", "nombre_modelo")

# Transicionar
client.transition_model_version_stage(
    name="modelo", version=1, stage="Production"
)
```

### Búsqueda:
```python
runs = mlflow.search_runs(
    experiment_ids=["id"],
    filter_string="metrics.accuracy > 0.9",
    order_by=["metrics.accuracy DESC"]
)
```

---

## 🎯 Resumen de Conceptos Clave

✅ **MLflow Tracking** = Registrar experimentos (params, metrics, artifacts)  
✅ **MLflow Models** = Empaquetar modelos en formato estándar  
✅ **Model Registry** = Gestionar ciclo de vida (Staging → Production)  
✅ **Autolog** = Tracking automático según framework  
✅ **Signature** = Define input/output del modelo  
✅ **Stages** = None → Staging → Production → Archived  

### Flujo Completo:

```
1. Experimentar → mlflow.start_run()
2. Registrar → log_param(), log_metric()
3. Guardar modelo → log_model()
4. Registrar en Registry → register_model()
5. Validar → Staging
6. Producción → Production
7. Monitorear → Buscar y comparar runs
```

---

**¡Felicidades! Has completado la introducción a MLflow** 🎉

Ahora estás listo para el siguiente notebook: **03_MLOps_Completo_Databricks.ipynb**  
donde integraremos todo lo aprendido en un workflow completo de MLOps.